**NASKA** (Established Artist) vs **DETTE** (Emerging Artist)

Is there a real difference in terms of song features?


First of alla extract Data from a Public Pop Punk Spotify Playlist with Spotify API.

Playlist Link: (https://open.spotify.com/playlist/5IMFrVKaonvz1AHWOhaX2A?si=d0066c9bce4641df) by **DrewLacroix**

In [10]:
!pip install spotipy
import csv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Set up Spotify credentials
client_id = '88347ddb41d340f19724bb9d371581fe'
client_secret = '72dddf2be3324a4c877daa00275dd049'

# Initialize Spotipy client
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Define the playlist ID
playlist_id = 'https://open.spotify.com/playlist/5IMFrVKaonvz1AHWOhaX2A?si=5c5f6f7c6dd540a7'

# Get playlist details
playlist = spotify.playlist(playlist_id)

# Extract playlist name
playlist_name = playlist['name']

# Extract tracks
tracks = playlist['tracks']
total_tracks = tracks['total']

# Create a list to store the extracted data
playlist_data = []

# Paginate through all tracks
while len(playlist_data) < total_tracks:
    # Iterate over tracks
    for item in tracks['items']:
        track = item['track']

        # Extract track details
        track_name = track['name']
        artist_name = track['artists'][0]['name']
        album_name = track['album']['name']
        track_id = track['id']

        # Get audio features of the track
        audio_features = spotify.audio_features([track_id])[0]

        # Extract desired audio features
        danceability = audio_features['danceability']
        energy = audio_features['energy']
        valence = audio_features['valence']
        tempo = audio_features['tempo']
        duration_ms = audio_features['duration_ms']

        # Create a dictionary with track details and features
        track_data = {
            'Track': track_name,
            'Artist': artist_name,
            'Album': album_name,
            'Danceability': danceability,
            'Energy': energy,
            'Valence': valence,
            'Tempo': tempo,
            'Duration (ms)': duration_ms
        }

        # Append track data to the playlist_data list
        playlist_data.append(track_data)

    # Check if there are more tracks to retrieve
    if tracks['next']:
        tracks = spotify.next(tracks)
    else:
        break

# Print the total number of tracks extracted
print(f"Total Tracks: {len(playlist_data)}")

# Save the playlist data to a CSV file
import csv

csv_file = 'spotify_playlist_data.csv'

with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=playlist_data[0].keys())
    writer.writeheader()
    writer.writerows(playlist_data)

print(f"Playlist data has been saved to '{csv_file}'.")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Total Tracks: 248
Playlist data has been saved to 'spotify_playlist_data.csv'.


Let's print a small part to inspect if Variables are fine.

In [34]:
import pandas as pd

# Define the file path
csv_file_path = '/content/spotify_playlist_data.csv'

# Read the CSV file using pandas
df = pd.read_csv(csv_file_path)

# Print the DataFrame
df.head()


,Track,Artist,Album,Danceability,Energy,Valence,Tempo,Duration (ms)
0,N.D.X.S.,xDiemondx,N.D.X.S.,0.596,0.950,0.369,77.010,163636
1,a Nessuno,NASKA,La mia stanza,0.484,0.527,0.356,152.758,193306
2,Stupendo,Rino Castel,Stupendo,0.647,0.893,0.479,89.983,146333
3,B!TCH,Frasca,EMOTIVO,0.733,0.784,0.579,130.007,188308
4,SANGUE! (feat. Decrow),Frasca,SANGUE! (feat. Decrow),0.559,0.807,0.185,91.419,166154


**Quality Evaluation**

*   Number of Raws (They should be 248 as the number of Songs in this Playlist)

*   Missing Values (To see if the data was extracted correctly and to evaluate its Completeness)

*   Some Descriptive Statistics

*   Correlation Matrix





In [12]:
# Check the dimensions of the DataFrame
print(f"Number of rows: {df.shape[0]}")
print(f"Number of columns: {df.shape[1]}")
print()

# Check for missing values
missing_values = df.isnull().sum()
print("Missing Values:")
print(missing_values)
print()

# Check the data types of each column
data_types = df.dtypes
print("Data Types:")
print(data_types)
print()

# Descriptive statistics
statistics = df.describe()
print("Descriptive Statistics:")
print(statistics)
print()

# Check the correlation between numeric columns
numeric_columns = ['Danceability', 'Energy', 'Valence', 'Tempo', 'Duration (ms)']
correlation = df[numeric_columns].corr()
print("Correlation Matrix:")
print(correlation)


Number of rows: 248
Number of columns: 8

Missing Values:
Track            0
Artist           0
Album            0
Danceability     0
Energy           0
Valence          0
Tempo            0
Duration (ms)    0
dtype: int64

Data Types:
Track             object
Artist            object
Album             object
Danceability     float64
Energy           float64
Valence          float64
Tempo            float64
Duration (ms)      int64
dtype: object

Descriptive Statistics:
       Danceability      Energy     Valence       Tempo  Duration (ms)
count    248.000000  248.000000  248.000000  248.000000     248.000000
mean       0.531052    0.818790    0.519376  132.371528  171961.145161
std        0.103289    0.128592    0.184479   36.022948   27918.279442
min        0.134000    0.244000    0.035200   72.078000   96120.000000
25%        0.467000    0.751500    0.389250   94.975000  153328.250000
50%        0.530000    0.854000    0.521500  138.938000  171164.500000
75%        0.601250    0.921



*   Completness of Data is Satisfied

*   There are no significant Correlations between Variables (Features of a Song)



**Top 10 Songs in Playlist**

Let's define metrics to establish a Top 10 Songs in Playlist.

Suppose we're looking for **Danceability** over other features, it will have a greater weight. So the metric is defined by the sum of all feature multiplied by its weight.

In [37]:
# Define the weights for each audio feature (adjust according to preference)
weights = {
    'Danceability': 0.6,
    'Energy': 0.3,
    'Valence': 0.3
}

# Calculate the weighted score for each track
df['Score'] = df['Danceability'] * weights['Danceability'] + \
              df['Energy'] * weights['Energy'] + \
              df['Valence'] * weights['Valence']

# Sort the DataFrame by the score in descending order
df_sorted = df.sort_values('Score', ascending=False)

# Get the top 50 tracks
top_50_tracks = df_sorted.head(50)

# Print the top 50 tracks
top_50_tracks.head()


,Track,Artist,Album,Danceability,Energy,Valence,Tempo,Duration (ms),Score
51,Polly,NASKA,REBEL,0.737,0.782,0.931,132.054,165509,0.9561
207,Punk,Iride,Punk,0.682,0.806,0.925,77.492,139258,0.9285
11,Sex Love Punk Rock,Revel,Viva La Revelution! -Volume 1-,0.649,0.959,0.757,91.552,169672,0.9042
239,Nella mia testa,xDiemondx,Nella mia testa,0.672,0.850,0.784,128.013,217500,0.8934
5,Riposa in pace,Fløp,Riposa in pace,0.611,0.946,0.808,140.045,157775,0.8928


Once the metric for each song has been calculated, it's possible to sort the songs and plot them.

In [21]:
import plotly.graph_objects as go
# Group the DataFrame by artist and track and calculate the mean score for each group
df_grouped = df.groupby(['Artist', 'Track']).mean().reset_index()

# Sort the DataFrame by the mean score in descending order
df_sorted = df_grouped.sort_values('Score', ascending=False)

# Get the top 10 artists
top_10_artists = df_sorted.groupby('Artist').head(1).nlargest(10, 'Score')

# Create the interactive bar plot
fig = go.Figure(data=[go.Bar(x=top_10_artists['Artist'], y=top_10_artists['Score'],
                            text=top_10_artists['Track'],
                            hovertemplate='<b>Artist</b>: %{x}<br><b>Score</b>: %{y}<br><b>Track</b>: %{text}<extra></extra>')])

fig.update_layout(title='Top 10 Artists and Their Top Tracks',
                  xaxis_title='Artist',
                  yaxis_title='Score')

fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6)

fig.show()



<ipython-input-21-c9454ffc26f9>:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_grouped = df.groupby(['Artist', 'Track']).mean().reset_index()


Plot is interactive: By selecting each Song in the plot you can see the score recieved.

**NASKA vs. Dette**

Since Dette is featured with only one song in this playlist, let's compare her "CRETEEN 2.0" with all of NASKA's songs featured in this playlist.

In [31]:
import plotly.express as px

# Filter the DataFrame for NASKA and Dette
naska_df = df[df['Artist'] == 'NASKA']
dette_df = df[df['Artist'] == 'Dette']

# Combine the DataFrames for NASKA and Dette
combined_df = pd.concat([naska_df, dette_df])

# Calculate the mean of each feature for NASKA and Dette
mean_features_df = combined_df.groupby('Artist').mean().reset_index()

# Reshape the DataFrame for plotting
melted_df = pd.melt(mean_features_df, id_vars='Artist', value_vars=['Danceability', 'Energy', 'Valence'],
                    var_name='Feature', value_name='Value')

# Merge the melted DataFrame with the original DataFrame to get the track titles
merged_df = pd.merge(melted_df, df[['Artist', 'Track']], on='Artist', how='left')

# Create the interactive bar plot
fig = px.bar(merged_df, x='Feature', y='Value', color='Artist', barmode='group',
             text='Track', labels={'Feature': 'Feature', 'Value': 'Value'},
             title='Comparison of Features: NASKA vs Dette')

fig.update_traces(textposition='outside')

fig.show()


<ipython-input-31-595778f2a901>:11: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



Plot is Interactive: By selecting each stack of NASKA bars you can see the score for each song.

Clearly NASKA has far more songs than Dette and as a result the comparison doesn't hold up.

**NASKA vs. Dette In Deep**

But what if we averaged his scores?

In [33]:
# Calculate the mean of each feature for NASKA and Dette
mean_features_df = combined_df.groupby('Artist').mean().reset_index()

# Reshape the DataFrame for plotting
melted_df = pd.melt(mean_features_df, id_vars='Artist', value_vars=['Danceability', 'Energy', 'Valence'],
                    var_name='Feature', value_name='Value')

# Create the interactive scatter plot
fig = px.scatter(melted_df, x='Feature', y='Value', color='Artist', hover_data=['Artist'],
                 labels={'Feature': 'Feature', 'Value': 'Value'},
                 title='Comparison of Features: NASKA vs Dette')

fig.update_traces(marker=dict(size=10, opacity=0.8))

fig.show()

<ipython-input-33-c51bf11c9e03>:2: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



By averaging all the scores of the different NASKA songs and comparing the average value with Dette's scores for the different features, we notice that something has changed.

Dette has a big lead on all features.

**Conclusions**

Definitely not a super fair method of comparison for the following reasons:



*   Dette could be present in the playlist with a single piece, her best song for example and consequently have objective statistics higher than the average ones of NASKA, the fact remains that NASKA is still a great artist, superior to Dette. Clearly if we were to average every Dette score against NASKA's scores, the up-and-coming artist wouldn't stand a chance.

*   The metric used takes more account of one feature rather than the other two, making it weigh more. It is certainly not a good method to decree which of the two is the better artist. This statistic belongs to the public!





In conclusion, we can say that NASKA is constant in its publications receiving a very high average score for each song, while for Dette we can say that if it were to continue as it started with "CRETEEN 2.0" it would put the established artist in serious difficulty!